In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
!pip install --upgrade -q kaggle

!mkdir /root/.kaggle
import json
token = {
    "username": "zahidhussain909",
    "key": "39a06efd89d0f2a699143b8d3d62b216"
}

with open('/root/.kaggle/kaggle.json', 'w') as config_file:
    json.dump(token, config_file)
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d captaintyping/healthcare-domain-dataset

  0% 0.00/29.6k [00:00<?, ?B/s]
100% 29.6k/29.6k [00:00<00:00, 53.8MB/s]


In [ ]:
import zipfile
zipfile=zipfile.ZipFile('/content/healthcare-domain-dataset.zip','r')
zipfile.extractall()
zipfile.close()

In [ ]:
train = pd.read_csv('Training.csv')
test = pd.read_csv('Testing.csv')

In [ ]:
X =  train.drop("prognosis" , axis=1)
y = train.prognosis

In [ ]:
dimensionality_reduction = train.groupby(train['prognosis']).max()

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
cols     = train.columns
cols     = cols[:-1]

In [ ]:
imp_features = classifier.feature_importances_
indices = np.argsort(imp_features)[::-1]
features = cols

In [ ]:
from sklearn.tree import _tree

In [ ]:
def execute_bot():

    print("Please reply with yes/Yes or no/No for the following symptoms") 
    def print_disease(node):
        #print(node)
        node = node[0]
        #print(len(node))
        val  = node.nonzero() 
        #print(val)
        disease = labelencoder.inverse_transform(val[0])
        return disease
    def tree_to_code(tree, feature_names):
        tree_ = tree.tree_
        #print(tree_)
        feature_name = [
            feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
            for i in tree_.feature
        ]
        #print("def tree({}):".format(", ".join(feature_names)))
        symptoms_present = []
        def recurse(node, depth):
            indent = "  " * depth
            if tree_.feature[node] != _tree.TREE_UNDEFINED:
                name = feature_name[node]
                threshold = tree_.threshold[node]
                print(name + " ?")
                ans = input()
                ans = ans.lower()
                if ans == 'yes':
                    val = 1
                else:
                    val = 0
                if  val <= threshold:
                    recurse(tree_.children_left[node], depth + 1)
                else:
                    symptoms_present.append(name)
                    recurse(tree_.children_right[node], depth + 1)
            else:
                present_disease = print_disease(tree_.value[node])
                print( "You may have " +  present_disease )
                print()
                red_cols = dimensionality_reduction.columns 
                symptoms_given = red_cols[dimensionality_reduction.loc[present_disease].values[0].nonzero()]
                print("symptoms present  " + str(list(symptoms_present)))
                print()
                print("symptoms given "  +  str(list(symptoms_given)) )  
                print()
                confidence_level = (1.0*len(symptoms_present))/len(symptoms_given)
                print("confidence level is " + str(confidence_level))
                print()
                print('The model suggests:')
                print()
                row = doctors[doctors['disease'] == present_disease[0]]
                print('Consult ', str(row['name'].values))
                print()
                print('Visit ', str(row['link'].values))
                #print(present_disease[0])
                
    
        recurse(0, 1)
    
    tree_to_code(classifier,cols)

In [ ]:
!wget https://raw.githubusercontent.com/aryanveturekar/Health-Care-Chat-Bot/master/doctors_dataset.csv

--2021-08-27 19:16:28--  https://raw.githubusercontent.com/aryanveturekar/Health-Care-Chat-Bot/master/doctors_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6048 (5.9K) [text/plain]
Saving to: ‘doctors_dataset.csv’

doctors_dataset.csv 100%[===================>]   5.91K  --.-KB/s    in 0s      

2021-08-27 19:16:28 (78.1 MB/s) - ‘doctors_dataset.csv’ saved [6048/6048]



In [ ]:
doc_dataset = pd.read_csv('doctors_dataset.csv', names = ['Name', 'Description'])


diseases = dimensionality_reduction.index
diseases = pd.DataFrame(diseases)

doctors = pd.DataFrame()
doctors['name'] = np.nan
doctors['link'] = np.nan
doctors['disease'] = np.nan

doctors['disease'] = diseases['prognosis']


doctors['name'] = doc_dataset['Name']
doctors['link'] = doc_dataset['Description']

record = doctors[doctors['disease'] == 'AIDS']
record['name']
record['link']

1    https://www.practo.com/delhi/doctor/dr-54-gene...
Name: link, dtype: object

In [ ]:
execute_bot()

Please reply with yes/Yes or no/No for the following symptoms
palpitations ?
no 
pain_behind_the_eyes ?
no
receiving_blood_transfusion ?
no
red_spots_over_body ?
yes
['You may have Chicken pox']

symptoms present  ['red_spots_over_body']

symptoms given ['itching', 'skin_rash', 'fatigue', 'lethargy', 'high_fever', 'headache', 'loss_of_appetite', 'mild_fever', 'swelled_lymph_nodes', 'malaise', 'red_spots_over_body']

confidence level is 0.09090909090909091

The model suggests:

Consult  ['Dr. Inderjeet Singh']

Visit  ['https://www.practo.com/delhi/doctor/inderjeet-singh-ayurveda-sexologist?specialization=Homoeopath&practice_id=1219975']
